#Automatic Differentiation

Automatic Differentiation is a core tool used to calculate derivatives which are key to machine learning. This is my personal implementation.

Eric Buehler 2023

In [ ]:
#https://towardsdatascience.com/build-your-own-automatic-differentiation-program-6ecd585eec2a
#https://e-dorigatti.github.io/math/deep%20learning/2020/04/07/autodiff.html
#https://jingnanshi.com/blog/autodiff.html

In [ ]:
from abc import ABC, abstractmethod
from typing import *
from functools import reduce
import numpy as np

In [ ]:
class DifferentiableValue(ABC):
    count = 0
    def __init__(self):
        DifferentiableValue.count += 1
        self.id = DifferentiableValue.count 

    @abstractmethod
    def backward(self, var):
        pass

    @abstractmethod
    def forward(self):
        pass

    @abstractmethod
    def __repr__(self) -> str:
        pass

In [ ]:
class Graph:
    def __init__(self):
        self.values = []
        global _graph
        _graph = self

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        pass

    def forward(self):
        return self.values[0].forward()

    def backward(self):
        graph = self.values.copy()
        res = self.values[0].backward()
        self.values = graph
        return res

    def get_gradient(self) -> List[Any]:
        return list(filter(lambda item: item != None, self.values[0].get_gradient()))

In [ ]:
def remove_copies(inputs = List[DifferentiableValue]):
    for input in inputs:
        for i, value in enumerate(_graph.values):
            if value.id == input.id:
                _graph.values.pop(i)

def generate_topo(graph: DifferentiableValue) -> List[DifferentiableValue]:
    topo = []
    visited = set()
    def build_topo(node: DifferentiableValue) -> List[DifferentiableValue]:
        if node not in visited:
            visited.add(node)
            if hasattr(node, "inputs"):
                for input in node.inputs:
                    build_topo(input)
            topo.append(node)
        return topo
    return build_topo(graph)

In [ ]:
class Constant(DifferentiableValue):
    count = 0

    def __init__(self, value, name = None):
        super().__init__()
        _graph.values.append(self)
        self.value = value
        self.name = name
        Constant.count += 1
        
        self.gradient = 0
        
    def backward(self):
        self.gradient = 0
        
    def forward(self) -> Any:
        return self.value

    def get_gradient(self) -> List[Any]:
        return [self.gradient]

    def __repr__(self) -> str:
        return f"Constant('{self.name}' {self.value}, g={self.gradient})"

class Variable(DifferentiableValue):
    count = 0

    def __init__(self, value = None, name = None):
        super().__init__()
        _graph.values.append(self)
        self._value = value
        self.name = name
        Variable.count += 1
        
        self.gradient = 0
        
    def backward(self):
        self.gradient = 1
    
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, value):
        if self.value == None:
            raise ValueError("Variable does not have value")
        self._value = value
        
    def forward(self) -> Any:
        if self.value == None:
            raise ValueError("Variable does not have value")
        return self.value

    def get_gradient(self) -> List[Any]:
        return [self.gradient]

    def __repr__(self) -> str:
        return f"Variable('{self.name}' {self.value}, g={self.gradient})"

In [ ]:
class Sum(DifferentiableValue):
    count = 0

    def __init__(self, left: DifferentiableValue, right:  DifferentiableValue):
        super().__init__()
        remove_copies([left, right])
        _graph.values.append(self)
        self.inputs = [left, right]
        Sum.count += 1
        
        self.gradient = 0

        def _backward():
            for input in self.inputs:
                input.gradient += self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
        
    def forward(self) -> SupportsFloat:
        return sum([input.forward() for input in self.inputs])
    
    def __repr__(self) -> str:
        return "Sum({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Product(DifferentiableValue):
    count = 0

    def __init__(self, left: DifferentiableValue, right:  DifferentiableValue):
        super().__init__()
        remove_copies([left, right])
        _graph.values.append(self)
        self.inputs = [left, right]
        Product.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += self.inputs[1].forward() * self.gradient
            self.inputs[1].gradient += self.inputs[0].forward() * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
        
    def forward(self) -> SupportsFloat:
        return reduce((lambda x, y: x * y), [input.forward() for input in self.inputs])

    def __repr__(self) -> str:
        return "Product({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Power(DifferentiableValue):
    count = 0

    def __init__(self, base: DifferentiableValue, pow:  DifferentiableValue):
        super().__init__()
        remove_copies([base, pow])
        _graph.values.append(self)
        self.inputs = [base, pow]
        Power.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += (self.inputs[1].forward() * self.inputs[0].forward() ** (self.inputs[1] - 1).forward()) * self.gradient
            self.inputs[1].gradient += np.log(self.inputs[0].forward()) * self.inputs[0].forward() ** (self.inputs[1].forward()) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
        
    def forward(self) -> SupportsFloat:
        return reduce((lambda x, y: x ** y), [input.forward() for input in self.inputs])

    def __repr__(self) -> str:
        return "Power({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

In [ ]:
def generate_operation(op, self, other):
    if isinstance(other, DifferentiableValue):
        return op(self, other)
    if isinstance(other, (SupportsFloat)):
        return op(self, Constant(other))
    raise TypeError(f"Incompatible type for operation: {type(other)}.")

DifferentiableValue.__add__ = lambda self, other: generate_operation(Sum, self, other)
DifferentiableValue.__sub__ = lambda self, other: self + -other
DifferentiableValue.__neg__ = lambda self: self * -1
DifferentiableValue.__mul__ = lambda self, other: generate_operation(Product, self, other)
DifferentiableValue.__pow__ = lambda self, other: generate_operation(Power, self, other)
DifferentiableValue.__truediv__ = lambda self, other: self * (other ** -1)

In [ ]:
with Graph() as graph:
    x = Variable(2, "x")
    y = x**2
    
print("Raw Graph:")
print(graph.values)
print()

print("Topological Graph:")
topo = generate_topo(y)
for item in topo:
    print(item)
print()

print("Forward value:")
print(graph.forward())
print()

print("Backward graph:")
graph.backward()
print(graph.values)
print()

del graph

Raw Graph:
[Power(Variable('x' 2, g=0), Constant('None' 2, g=0), g=0)]

Topological Graph:
Variable('x' 2, g=0)
Constant('None' 2, g=0)
Power(Variable('x' 2, g=0), Constant('None' 2, g=0), g=0)

Forward value:
4

Backward graph:
[Power(Variable('x' 2, g=4), Constant('None' 2, g=2.772588722239781), g=1)]

